In [1]:
import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.60:4041
SparkContext available as 'sc' (version = 3.0.0-preview2, master = local[*], app id = local-1591997419201)
SparkSession available as 'spark'


import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils


In [7]:
val data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt")
data

data: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[55] at map at MLUtils.scala:86
res2: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[55] at map at MLUtils.scala:86


In [8]:
val splits = data.randomSplit(Array(0.7, 0.3), seed = 11L)
val training = splits(0).cache()
val test = splits(1)

splits: Array[org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint]] = Array(MapPartitionsRDD[56] at randomSplit at <console>:32, MapPartitionsRDD[57] at randomSplit at <console>:32)
training: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[56] at randomSplit at <console>:32
test: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[57] at randomSplit at <console>:32


In [9]:
val model = new LogisticRegressionWithLBFGS()
  .setNumClasses(10)
  .run(training)

model: org.apache.spark.mllib.classification.LogisticRegressionModel = org.apache.spark.mllib.classification.LogisticRegressionModel: intercept = 0.0, numFeatures = 6228, numClasses = 10, threshold = 0.5


In [10]:
val predictionAndLabels = test.map { case LabeledPoint(label, features) =>
  val prediction = model.predict(features)
  (prediction, label)
}

predictionAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[95] at map at <console>:32


In [16]:
val metrics = new MulticlassMetrics(predictionAndLabels)
val accuracy = metrics.accuracy

println(s"Accuracy = $accuracy")

Accuracy = 1.0


metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@3b155e81
accuracy: Double = 1.0


In [12]:
model.save(sc, "target/tmp/scalaLogisticRegressionWithLBFGSModel")

val sameModel = LogisticRegressionModel.load(
  sc,
  "target/tmp/scalaLogisticRegressionWithLBFGSModel"
)

sameModel: org.apache.spark.mllib.classification.LogisticRegressionModel = org.apache.spark.mllib.classification.LogisticRegressionModel: intercept = 0.0, numFeatures = 6228, numClasses = 10, threshold = 0.5


In [13]:
sameModel

res4: org.apache.spark.mllib.classification.LogisticRegressionModel = org.apache.spark.mllib.classification.LogisticRegressionModel: intercept = 0.0, numFeatures = 6228, numClasses = 10, threshold = 0.5
